In [4]:
from __future__ import print_function
import cv2 as cv
import argparse
import numpy as np

In [10]:
def pre_process(dir, file_name, folder="", method="KNN", filter_color=False):
    """Process each individual frame of a video, generating a high contrast set of frames.
    It removes the background and light reflections on the surface of the fluid, with an optional feature of removing the color.
    It's configured to filter all non-blue color by default. To remove another, modify the lower_color and upper_color variables."""
    
    ### Choosing method selected for the background substractor object
    if method == "MOG2":
        backSub = cv.createBackgroundSubtractorMOG2(detectShadows=False)
    if method == "KNN":
        backSub = cv.createBackgroundSubtractorKNN(detectShadows=False)
    else:
        raise(Exception("Invalid method name"))
    
    capture = cv.VideoCapture(cv.samples.findFileOrKeep(dir+"/"+file_name))

    i = 0 ### Initialazing iterable variable
     
    Masks = [] ### Initialazing mask list for saving each mask (i.e. Image matrix comprised of 0s and 255s, black and white.)
    if not capture.isOpened():
        raise(Exception("Unable to open file"))
    while True:
        ret, frame = capture.read() ### Reads each frame of the video
        if frame is None:
            break
        
        fgMask = backSub.apply(frame) ### Substract background
        
        cv.rectangle(frame, (10, 2), (100,20), (255,255,255), -1)
        cv.putText(frame, str(capture.get(cv.CAP_PROP_POS_FRAMES)), (15, 15),
        cv.FONT_HERSHEY_SIMPLEX, 0.5 , (0,0,0))
        


        res1 = cv.bitwise_and(frame, frame, mask=fgMask)

        gray = cv.cvtColor(res1, cv.COLOR_BGR2GRAY)

        # threshold grayscale image to extract glare
        mask = cv.threshold(gray, 220, 255, cv.THRESH_BINARY)[1]
        res2 = cv.inpaint(res1, mask, 21, cv.INPAINT_TELEA) 

        if filter_color:

            ### convert the BGR image to HSV colour space
            hsv = cv.cvtColor(res2, cv.COLOR_BGR2HSV)

            ### set the lower and upper bounds for the color hue
            lower_color = np.array([60, 35, 140])
            upper_color = np.array([180, 255, 255])
 
            ### create a mask for color colour using inRange function
            mask = cv.inRange(hsv, lower_color, upper_color)

            ### perform bitwise and on the original image arrays using the mask
            res3 = cv.bitwise_and(res2, res2, mask=mask)
            
            ### display original frame, and filtered black and white frame. 
            cv.imshow('Frame', frame)
            cv.imshow('FG Mask', mask)

            if i < 10:
                imagen_numero = f"000{i}.jpg"
            elif 100 > i >= 10:
                imagen_numero = f"00{i}.jpg"
            else:
                imagen_numero = f"0{i}.jpg"

            ### Save file
            cv.imwrite(dir+folder+"/"+imagen_numero,res3)
            i += 1

        else:
            ### display original frame, and filtered frame. 
            cv.imshow('Frame', frame)
            cv.imshow('FG Mask', res2)

            if i < 10:
                imagen_numero = f"000{i}.jpg"
            elif 100 > i >= 10:
                imagen_numero = f"00{i}.jpg"
            else:
                imagen_numero = f"0{i}.jpg"
            
            ### Save file
            cv.imwrite(dir+folder+"/"+imagen_numero,res2)
            i += 1
        
        Masks.append(mask) ### Add mask to mask list
        keyboard = cv.waitKey(30)
        if keyboard == 'q' or keyboard == 27:
            break
        
        ### Display progress
        if i%20 == 0:
            print(i)
    return Masks

### Particular example

In [11]:
dir = "/Users/eitu/Documents/Facu/Labo5/Con colorante sin luces glitter"
folder = "/prueba PIV casero jpg"
file_name = "IMG_0239.mov"
method = "KNN"

Masks = pre_process(dir=dir, file_name=file_name, folder=folder, method=method, filter_color=True)

20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640


In [ ]:
### Close all windows
capture.release()
video.release()
cv.destroyAllWindows()
cv.waitKey(1)

### FALTA PIV manual

In [89]:
test_mask = Masks[42]
test_mask = test_mask/255
test_mask = test_mask.astype(int)

test_mask_islands = np.zeros_like(test_mask)

for i in range(test_mask.shape[0]-1):
    for j in range(test_mask.shape[1]-1):
        if test_mask[i,j] == 1:
            test_mask_islands[i, j] = test_mask[i+1, j] + test_mask_islands[i-1, j] + test_mask_islands[i, j+1] + test_mask_islands[i, j-1]

print(test_mask)
print("------")
print(test_mask_islands)

# def find_centers(mask):

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 1 1 1]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
------
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 2 2 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
